<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/Coil3D_FrenetSerret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots

In [2]:
theta=np.linspace(0,2*np.pi,36)
xy=np.array([np.cos(theta),np.sin(theta)])

In [3]:
xy_stretch=np.array([[2.,0.],[0.,1.]]) @ xy

rot_angle = np.pi/6
xy_stretch_rotate = np.array([[np.cos(rot_angle), -np.sin(rot_angle)],[np.sin(rot_angle), np.cos(rot_angle)]]) @ xy_stretch

In [4]:
fig=make_subplots()
fig.add_scatter(x=xy[0],y=xy[1],mode='markers')
fig.add_scatter(x=xy_stretch[0],y=xy_stretch[1],mode='markers')
fig.add_scatter(x=xy_stretch_rotate[0],y=xy_stretch_rotate[1],mode='markers')
fig.update_xaxes(range=[-3,3])
fig.update_yaxes(range=[-3,3])
fig.update_layout(width=600,height=600,template='plotly_dark')

In [5]:
t=np.linspace(0,4*np.pi,360)
R=3.
mz=0.4

In [6]:
def curve(t, R, mz):
    xyz = np.array([R*np.cos(t), R*np.sin(t), mz*t])
    return xyz

In [7]:
xyz= curve(t,R,mz)

In [ ]:
fig=make_subplots(rows=1,cols=1,specs=[[{'type':'surface'}]])
fig.add_scatter3d(x=xyz[0,:],y=xyz[1,:],z=xyz[2,:],mode='lines',row=1,col=1)
fig.update_layout(width=600,height=600,template='plotly_dark')

In [28]:
def get_grad(f):

    def grad(x,h):
        res=[]
        xplush=np.copy(x)
        for i in range(x.size):
            xplush[i]=x[i]+h
            res.append((f(xplush) - f(x))/h)
        return np.array(res)
    return grad
        



In [29]:
def f(x):
    return np.sin(x[0])+ np.cos(2*x[1])

In [30]:
grad=get_grad(f)

In [31]:
grad(np.array([np.pi/6, np.pi/3]),h=1e-8)

array([ 0.8660254 , -0.86602539])